# Introduction

When I started to study reinforcement learning I did not find any good online resource which explained from the basis what reinforcement learning really is. Most of the (very good) blogs out there focus on the modern approaches (Deep Reinforcement Learning) and introduce the **Bellman equation** without a satisfying explanation. I turned my attention to books and I found the one of **Russel and Norvig** called **Artificial Intelligence: A Modern Approach**.

<img src="files/figures/artificial_intelligence_a_modern_approach.png" style="width: 500px;" />

This article is based on **chapter 17** of the second edition, and it can be considered an extended review of the chapter. I will use the same mathematical notation of the authors, in this way you can use the book to cover some missing parts or vice versa. In the next section I will introduce **Markov chain**, if you have already know this concept you can skip to the next section...

# In the Beginning was Andrey Markov

**Andrey Markov** was a Russian mathematician who studied stochastic processes. Markov was particularly interested in systems that follows a chain of linked events. In 1906 Markov produced interesting results about discrete processes that he called **chain**. A **Markov chain** has a set of **states** $S=\{ s_0, s_1, \ldots, s_m \}$ and a **process** that can move successively from one state to another. Each move is a single **step** and is based on a **transition model** $T$. You should make some effort in remembering the keywords in bold because we will use them extensively during the rest of the article. To summarise a Markov chain is defined by:

1. Set of possible states: $S=\{ s_0, s_1, \ldots, s_m \}$
2. Initial state: $s_0$
3. Transition Model: $T(s, s')$

There is something peculiar in a Markov chain that I did not mention. A Markov chain is based on the **Markov Property**. The Markov property states that **given the present, the future is conditionally independent of the past**. That's it, the state in which the process is now it is dependent only from the state it was at $t-1$. An example can simplify the digestion of Markov chains. Let's suppose we have a chain with only two states $s_0$ and $s_1$, where $s_0$ is the initial state. The process is in $s_0$ 90% of the time and it can move to $s_1$ the remaining 10% of the time. When the process is in state $s_1$ it will remain there 50% of the time. Given this data we can create a **Transition Matrix** $T$ as follow:

\begin{equation}
T =
\begin{bmatrix}
   0.90 & 0.10 \\
   0.50 & 0.50
\end{bmatrix}
\end{equation}

The transition matrix is always a square matrix, and since we are dealing with probability distributions all the entries are within 0 and 1 and a single row sums to 1. **We can graphically represent the Markov chain**. In the following representation each state of the chain is a node and the transition probabilities are edges. Highest probabilities have a thickest edge:

<img src="files/figures/simple_markov_chain.png" style="width: 500px;" />

Until now we did not mention **time**, but we have to do it because Markov chains are dynamical processes which evolve in time. Let's suppose we have to guess where the process will be after 3 steps and after 50 steps. How can we do it? We are interested in chains that have a finite number of states and are time-homogeneous meaning that the transition matrix does not change over time. Given these assumptions **we can compute the k-step transition probability as the k-th power of the transition matrix**, let's do it in Numpy:

```python
import numpy as np

# Declaring the Transition Matrix T
T = np.array([[0.90, 0.10],
             [0.50, 0.50]])

# Obtaining T after 3 steps
T_3 = np.linalg.matrix_power(T, 3)
# Obtaining T after 50 steps
T_50 = np.linalg.matrix_power(T, 50)
# Obtaining T after 100 steps
T_100 = np.linalg.matrix_power(T, 100)

# Printing the matrices
print("T: " + str(T))
print("T_3: " + str(T_3))
print("T_50: " + str(T_50))
print("T_100: " + str(T_100))
```

```
T: [[ 0.9  0.1]
    [ 0.5  0.5]]

T_3: [[ 0.844  0.156]
      [ 0.78   0.22 ]]

T_50: [[ 0.83333333  0.16666667]
       [ 0.83333333  0.16666667]]

T_100: [[ 0.83333333  0.16666667]
        [ 0.83333333  0.16666667]]
```

Now we define the **initial distribution** which represent the state of the system at k=0. Our system is composed of two states and we can model the initial distribution as a vector with two elements, the first element of the vector represents the probability of staying in the state $s_0$ and the second element the probability of staying in state $s_1$. Let's suppose that we start from $s_0$, the vector $\boldsymbol{v}$ representing the initial distribution will have this form:

$$\boldsymbol{v} = (1, 0)$$

We can calculate **the probability of being in a specific state after k iterations** multiplying the initial distribution and the transition matrix: $\boldsymbol{v}\cdot T^k$. Let's do it in Numpy:

```python
import numpy as np

# Declaring the initial distribution
v = np.array([[1.0, 0.0]])
# Declaring the Transition Matrix T
T = np.array([[0.90, 0.10]
              [0.50, 0.50]])

# Obtaining T after 3 steps
T_3 = np.linalg.matrix_power(T, 3)
# Obtaining T after 50 steps
T_50 = np.linalg.matrix_power(T, 50)
# Obtaining T after 100 steps
T_100 = np.linalg.matrix_power(T, 100)

# Printing the initial distribution
print("v: " + str(v))
print("v_1: " + str(np.dot(v, T)))
print("v_3: " + str(np.dot(v, T_3)))
print("v_50: " + str(np.dot(v, T_50)))
print("v_100: " + str(np.dot(v, T_100)))
```

```
v: [[ 1.  0.]]

v_1: [[ 0.9  0.1]]

v_3: [[ 0.844  0.156]]

v_50: [[ 0.83333333  0.16666667]]

v_100: [[ 0.83333333  0.16666667]]
```

**What's going on?** The process starts in $s_0$ and after one iteration we can be 90% sure it is still in that state. This is easy to grasp, our transition model says that the process can stay in $s_0$ with 90% probability, nothing new. Looking to the state distribution at k=3 we noticed that there is something different. We are moving in the future and different braches are possible. If we want to find the probability of being in state $s_0$ after three iteration we should sum all the possible branches that lead to $s_0$. A picture is worth a thousand words:

<img src="files/figures/markov_chain_tree.png" style="width: 500px;" />

The possibility to be in $s_0$ at $k=3$ is given by (0.729 + 0.045 + 0.045 + 0.025) which is equal to 0.844 we got the same result. Now let's suppose that at the beginning we have some uncertainty about the starting state of our process, let's define another starting vector as follow:

$$\boldsymbol{v} = (0.5, 0.5)$$

That's it, with a probability of 50% we can start from $s_0$. Running again the Python script we print the results after 1, 3, 50 and 100 iterations:

```
v: [[ 0.5, 0.5]]

v_1: [[ 0.7  0.3]]

v_3: [[ 0.812  0.188]]

v_50: [[ 0.83333333  0.16666667]]

v_100: [[ 0.83333333  0.16666667]]
```

This time the probability of being in $s_0$ at $k=3$ is lower (0.812), but in the long run we have the same outcome (0.8333333). **What is happening in the long run?** The result after 50 and 100 iterations are the same and `v_50` is equal to `v_100` no matter which starting distribution we have. The chain **converged to equilibrium** meaning that as the time progresses it forgets about the starting distribution. But we have to be careful, the convergence is not always guaranteed. The dynamics of a Markov chain can be very complex, in particular it is possible to have **transient and recurrent states**.

# Markov Decision Process

In reinforcement learning it is often used a concept which is affine to Markov chain, I am talking about **Markov Decision Process (MDP)**. A MDP is a reinterpretation of Markov chains which includes an **agent** and a **decision making** process. A MDP is defined by these components:

1. Set of possible states: $S=\{ s_0, s_1, \ldots, s_m \}$
2. Initial state: $s_0$
3. Set of possible actions: $A=\{ a_0, a_1, \ldots, a_n \}$
4. Transition model: $T(s, a, s')$
5. Reward function: $R(s)$

As you can see we are introducing some new elements respect to Markov chains, in particular the transition model depends on the current state, the next state and the action of the agent. The transition model returns the probability of reaching the state $s'$ if the action $a$ is done in state $s$. But given $s$ and $a$ the model is conditionally independent of all previous states and actions (Markov Property). Moreover there is the **reward function** $R(s)$ which return a real value every time the agent moves from one state to the other (Attention: defining the reward function to depend only from $s$ can be confusing, Russel and Norvig used this notation in the book to simplify the description, it does not change the problem in any significant way). Since we have a reward function we can say that **some states are more desirable than others** because when the agent moves in those states it receives an higher reward. On the opposite there are **states that are not desirable at all**, because when the agent moves there it receives a negative reward.

* **Problem** the agent has to maximise the reward avoiding states which return negative values and choosing the one which return positive values.

* **Solution** find a **policy** $\pi (s)$ which returns the action with the highest reward.

The agent can try different policies but only one of those can be considered an **optimal policy**, denoted by $\pi^{\ast}$, which yields to the highest expected utility. It is time to introduce an example that I am going to use along all the articles. This example is inspired by the simple environment presented by Russel and Norvig in chapter 17.1 of their book. Let's suppose we have a **cleaning robot** which has to reach a charging station. Our simple world is a $4 \times 3$ matrix where the starting point $s_0$ is at $(1, 1)$, the charging station at $(4, 3)$, dangerous stairs at $(4, 2)$, and an obstacle at $(2, 2)$. **The robot has to find the best way to reach the charging station** (Reward +1) **and to avoid falling down the flight of stairs** (Reward -1). Every time the robot takes a decision it is possible to have the interference of a stochastic factor (ex. the ground is slippery, an evil cat is stinging the robot), which makes the robot diverge from the original path 20% of the time. If the robot decides to go ahead in 10% of the cases it will finish on the left and in 10% of the cases on the right state. If the robot hits the wall or the obstacle it will bounce back to the previous position. The main characteristics of this world are the following:

* Discrete time and space
* Fully observable
* Infinite horizon
* Known transition model

The environment is **fully observable**, meaning that **the robot always knows which state it is in**. The infinite horizon clause should be explained further. The **infinite horizon** means that **there is not a fixed time limit**. If the agent has a policy for going back and forth in the same two states, it will go on forever. This assumption does not mean that in every episode the agent has to pass for a series of infinite states. When one of the two terminal states is reached, the episode stops. A representation of this world and of the transition model is reported below. Be careful with the indexing used by Russell and Norvig, it can be confusing. They named each state of the world by the column and row, starting from the bottom-left corner.

<img src="files/figures/reinforcement_learning_simple_world.png" style="width: 500px;" />

I said that the aim of the robot is to find **the best way to reach the charging station**, but what does it mean **the best way**? Depending on the type of reward the robot is receiving for each intermediate state we can have different optimal policies $\pi^{\ast}$. Let's suppose we are programming the firmware of the robot. Based on the battery level we give a different reward at each time step. The rewards for the two terminal states remain the same (charger=+1, stairs=-1). The obstacle at $(2, 2)$ is not a valid state and therefore there is no reward associated to it. Given these assumptions we can have four different cases:

1. $R(s) \leq -1.6284$ **extremely low battery**
2. $-0.4278 \leq R(s) \leq -0.085$ **quite low battery**
3. $-0.0221 \leq R(s) \leq 0$ **slightly low battery**
4. $R(s) > 0$ **fully charged**

For each one of these conditions we can try to guess which policy the agent will choose. In the **extremely low battery** scenario the agent receives such an high punishment that it only wants to stop the pain as soon as possible. Life is so painful that falling down the flight of stairs is a good choice. In the **quite low battery** scenario the agent takes the shortest path to the charging station, it does not care about falling down. In the **slightly low battery** case the robot does not take risks at all and it avoids the stairs at cost of banging against the wall. Finally in the **fully charged** case the agent avoids both the exists and remain in a steady state receiving a positive reward at each time step.

<img src="files/figures/reinforcement_learning_four_policies.png" style="width: 500px;" />

Until now we know which kind of policies can emerge in specific environments with defined rewards, but there is still something I did not talk about: how can the agent choose the best policy?

# The Bellman Equation

The previous section terminated with a question: **how can the agent choose the best policy**? To give an answer to this question I will present the Bellman equation. First of all we have to find a way to compare two policies. We can use the reward given at each state to obtain a measure of the **utility of a state sequence**. We define the utility of the states **history** $h$ as:

$$U_h = R(s_0) + \gamma R(s_1) + \gamma^2 R(s_2) + \ldots + \gamma^n R(s_n)$$

The previous formula defines the **discounted rewards** of a state sequence, where $\gamma \in [0, 1]$ is called the **discount factor**. The discount factor describe the preference of the agent for the current rewards over future rewards. A discount factor of 1.0 yields to a degeneration of the previous formula to **additive rewards**. The discounted rewards are not the only way we can estimate the utility, but it is the one which gives less problems. For example in the case of an infinite sequence of states the discounted reward yields to a **finite utility** (using the sum of infinite series), moreover we can also compare infinite sequences using the average reward obtained per time step. **How to compare the utility of single states**? The utility $U(s)$ can be defined as:

$$U(s) = \mathbb{E}\left[ \sum_{t=0}^{\infty} \gamma^t R(s_t) \right]$$

Let's remember that the utility is defined with respect of a policy $\pi$ which for brevity I did not mention. Once we have the utilities how can we choose the best action for the next state? Using the **maximum expected utility** principle which says that a rational agent should choose an action that maximise its expected utility. We are a step closer to the Bellman equation. What we missed is to remember that the utility of a state $s$ is correlated with the utility of its neighbours at $s'$, meaning:

$$U(s)=R(s)+\gamma \max_a \sum_{s'} T(s,a,s')U(s')$$

We just derived the **Bellman equation**! Using the Bellman equation an agent can estimate the best action to take and find the optimal policy. Let's try to dissect this equation. **First**, the term $R(s)$ is something we have to add for sure in the equation. We are in state $s$ and we know the reward given for that state, the utility must take it into account. **Second**, notice that the equation is using the transition model $T$ which is multiplied times the utility of the next state $s'$. If you think about that it makes sense, a state which has a low probability to happen (like the 10% probability of moving on the left and on the right in our simplified world) will have a lowest weight in the summation.

To empirically test the Bellman equation we are going to use our cleaning robot in the simplified $4 \times 3$ environment. In this example the reward for each non-terminal state is $R(s)=-0.04$. We can imagine to have the utility values for each one of the states, **for the moment you do not need to know how we got these values, imagine they appeared magically**. In the same magical way we obtained the optimal policy for the world (to double-check if what we will obtain from the Bellman equation makes sense). This image is very important, keep it in mind.

<img src="files/figures/reinforcement_learning_example_r004.png" style="width: 500px;" />

In our example we suppose **the robot starts from the state (1, 1)**. Using the Bellman equation we have to **find which is the action with the highest utility** between UP, LEFT, DOWN and RIGHT. We do not have the optimal policy, but we have the transition model and the utility values for each state. You have to remember the two main rules of our environment. First, if the robot bounce on the wall it goes back to the previous state. Second, the selected action is executed only with a probability of 80% in accordance with the transition model. Instead of dealing with crude numbers I want to show you a graphical illustration of the possible outcomes:

<img src="files/figures/reinforcement_learning_simple_world_bellman_example.png" style="width: 500px;" />

For each possible outcome I reported the utility and the probability given by the transition model. This correspond to the first part of the Bellman equation. The next step is to calculate the **product between the utility and the transition probability, then sum up the value for each action**.

<img src="files/figures/reinforcement_learning_simple_world_bellman_example_2.png" style="width: 500px;" />

We found out that for state $(1, 1)$ **the action UP has the highest value**. This is in accordance with the optimal policy we magically got. This part of the Bellman equation returns the action that maximizes the expected utility of the subsequent state, which is what an optimal policy should do:

$$\pi^{\ast}(s)=\operatorname*{argmax}_a \sum_{s'} T(s, a, s')U(s')$$

Now we have all the elements and we can plug the values in the Bellman equation finding the utility of the state $(1, 1)$:

$$U(s_{11}) = -0.04 + 1.0 \times 0.7456 = 0.7056$$

The Bellman equation works! What we need is a **Python implementation** of the equation to use in our simulated world. We are going to use the same terminology of the previous sections. Our world has $4 \times 3 = 12$ possible states. The starting vector contains 12 values and the transition matrix is a huge $12 \times 12 \times 4$ matrix (12 starting states, 12 next states, 4 actions) where most of the values are zeros (we can move only from one state to its neighbours). I generated the transition matrix using a script and I saved it as a Numpy matrix. In the script I defined the function `return_state_utility()` which is an implementation of the Bellman equation. Using this function we are going to print the utility of the state $(1, 1)$ and check if it is the same we found previously:

```python
import numpy as np

def return_state_utility(v, T, u, reward, gamma):
    """Return the state utility
    
    @param v the state vector
    @param T transition matrix
    @param u utility vector
    @param reward for that state
    @param gamma discount factor
    @return the utility of the state
    """
    action_array = np.zeros(4)
    for action in range(0, 4):
        action_array[action] = np.sum(np.multiply(u, np.dot(v, T[:, :, action])))
    return reward + gamma * np.max(action_array)

def main():
    # Starting state vector
    # The agent starts from (1, 1)
    v = np.array([[0.0, 0.0, 0.0, 0.0,
                   0.0, 0.0, 0.0, 0.0,
                   1.0, 0.0, 0.0, 0.0]])
    
    # Transition matrix loaded from file
    # (It is too big to write here)
    T = np.load("T.npy")
    
    # Utility vector
    u = np.array([[0.812, 0.868, 0.918,   1.0,
                   0.762,   0.0, 0.660,  -1.0,
                   0.705, 0.655, 0.611, 0.388]])
    
    # Defining the reward for state (1, 1)
    reward = -0.04
    # Assuming that the discount factor is equal to 1.0
    gamma = 1.0
    
    # Use the Bellman equation to find the utility of state (1, 1)
    utility_11 = return_state_utility(v, T, u, reward, gamma)
    print("Utility of state (1, 1): " + str(utility))

if __name__ == "__main__":
    main()
```

```
Utility of state (1, 1): 0.7056
```

That's great, we obtained exactly the same value! Until now **we supposed that the utility values appeared magically**. Instead of using a magician we want to **find an algorithm to obtain these values**. There is a problem. For $n$ possible states there are $n$ Bellman equations, and each equation contains $n$ unknowns. Using any linear algebra package would be possible to solve these equations, the problem is that they are not linear because of the `max` operator. What to do? We can use the value iteration algorithm...

# The Value Iteration Algorithm

The Bellman equation is the core of the value iteration algorithm for solving a MDP.

# 初次编辑日期 (Initial Edit Date)

2018年6月4日

# 参考文献 (References)

[1] https://mpatacchiola.github.io/blog/2016/12/09/dissecting-reinforcement-learning.html

[2] http://setosa.io/blog/2014/07/26/markov-chains/index.html